# Préparation des données de défaillance

Chargement des donnnées

In [1]:
import pandas as pd
import numpy  as np

#Chargement des données
defaillance = pd.read_csv("../data/raw/PdM_failures.csv")
# transformer les dates dans le bon type
defaillance["datetime"] = pd.to_datetime(defaillance["datetime"])

defaillance.head()

,datetime,machineID,failure
0,2015-01-05 06:00:00,1,comp4
1,2015-03-06 06:00:00,1,comp1
2,2015-04-20 06:00:00,1,comp2
3,2015-06-19 06:00:00,1,comp4
4,2015-09-02 06:00:00,1,comp4


On décale les donnnées d'un jour, puisqu'on veut les prédire à l'avance

In [2]:
from datetime import timedelta
defaillance['datetime'] = defaillance['datetime']+timedelta(days=-1)
defaillance.head()

,datetime,machineID,failure
0,2015-01-04 06:00:00,1,comp4
1,2015-03-05 06:00:00,1,comp1
2,2015-04-19 06:00:00,1,comp2
3,2015-06-18 06:00:00,1,comp4
4,2015-09-01 06:00:00,1,comp4


In [3]:
#Création des indexes
defaillance = defaillance.set_index(['machineID', 'datetime'])
defaillance = pd.get_dummies(defaillance["failure"])
defaillance = defaillance.groupby(["machineID","datetime"]).sum()
defaillance["total_failure"]=defaillance.sum(axis = 1)

Toutes les defaillances sont enregistrées à 06:00H. On créée l'index pour chaque jour.

In [4]:
machines = pd.read_csv("../data/raw/PdM_machines.csv")

time_series = pd.date_range(start = "2015-01-01 06:00:00", end = "2016-01-01 06:00:00", freq="1D")
indexes = pd.MultiIndex.from_product([machines["machineID"].values,time_series.values], names=['machineID', 'datetime'])

On fait une jointure de ces indexes avec les données de maintenance

In [5]:
defaillance = pd.DataFrame(index = indexes).join(defaillance).fillna(0)

In [6]:
defaillance.head(20)

comp1  comp2  comp3  comp4  total_failure
machineID datetime                                                      
1         2015-01-01 06:00:00    0.0    0.0    0.0    0.0            0.0
          2015-01-02 06:00:00    0.0    0.0    0.0    0.0            0.0
          2015-01-03 06:00:00    0.0    0.0    0.0    0.0            0.0
          2015-01-04 06:00:00    0.0    0.0    0.0    1.0            1.0
          2015-01-05 06:00:00    0.0    0.0    0.0    0.0            0.0
          2015-01-06 06:00:00    0.0    0.0    0.0    0.0            0.0
          2015-01-07 06:00:00    0.0    0.0    0.0    0.0            0.0
          2015-01-08 06:00:00    0.0    0.0    0.0    0.0            0.0
          2015-01-09 06:00:00    0.0    0.0    0.0    0.0            0.0
          2015-01-10 06:00:00    0.0    0.0    0.0    0.0            0.0
          2015-01-11 06:00:00    0.0    0.0    0.0    0.0            0.0
          2015-01-12 06:00:00    0.0    0.0    0.0    0.0            0.0
          2015-01-13 06:00:00    0.0    0.0    0.0    0.0            0.0
          2015-01-14 06:00:00    0.0    0.0    0.0    0.0            0.0
          2015-01-15 06:00:00    0.0    0.0    0.0    0.0            0.0
          2015-01-16 06:00:00    0.0    0.0    0.0    0.0            0.0
          2015-01-17 06:00:00    0.0    0.0    0.0    0.0            0.0
          2015-01-18 06:00:00    0.0    0.0    0.0    0.0            0.0
          2015-01-19 06:00:00    0.0    0.0    0.0    0.0            0.0
          2015-01-20 06:00:00    0.0    0.0    0.0    0.0            0.0

In [7]:
defaillance.to_csv("../data/prepared_data/defaillance.csv")